In [145]:
import json
import os
import re
import MeCab
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.preprocessing import sequence
import collections

# import gensim.parsing.preprocessing
from gensim import corpora, matutils
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split

def clean_text(text):
    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text


def tokenize(text):
    mecabTagger = MeCab.Tagger('mecal-ipadic-neologd')
    word_list = []
    res = mecabTagger.parseToNode(text)
    while res:
        pos = res.feature.split(",")
        if pos[0] in ["名詞"]:
            if not pos[1] in ["代名詞", "固有名詞", "数", "非自立", "特殊"]:
                try:
                    word_list.append(res.surface)
                except UnicodeDecodeError:
                    pass
#                     print('デコードエラー→'+pos[0]+pos[1]+pos[2])
        res = res.next
    return word_list

def make_words_list(data):
    words_list = [clean_text(text) for text in data]
    words_list = [tokenize(text) for text in words_list]
    return words_list

project_dir = os.getcwd()
DATA_DIR = os.path.join(project_dir, 'data')
SAVE_DIR = model_path = os.path.join(project_dir, 'model/doc2vec.model')

In [169]:
from pymongo import MongoClient

# コネクション作成
client = MongoClient('localhost', 27017)
db = client['by_user_database']
# データベースからコレクションを取得
collection = db['horiemon_collection']
# tweetのIDの最大値を取得
doc = list(collection.find().sort("id", -1).limit(1000))
texts = [doc[i]['text'] for i in range(len(doc))]
words_list = make_words_list(texts)
trainings = [TaggedDocument(words = words,tags = [i]) for i,words in enumerate(words_list)]

# トレーニング
m = Doc2Vec(documents= trainings, dm = 1, size=300, window=8, min_count=10, workers=4)

# モデルのセーブ
if not os.path.exists(SAVE_DIR):
    m.save(SAVE_DIR)

# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load(SAVE_DIR)

私がプロデュースしているカロリー1/2の豆腐生チョコ毎日限定で販売しています。是非お買い上げください！ 抹茶・ラムレーズン味も！ 私の情報は https://t.co/4OjgY4iFuh に集約されています。


In [199]:
db = client['#起業']
tweeter_list = db.collection_names()

for i in range(len(tweeter_list)):
    tweets = ''
    collection = db[tweeter_list[i]]
    doc = list(collection.find().sort("id", -1))
    for j in range(len(doc)):
        tweets = tweets + clean_text(doc[j]['text']) + os.linesep
    tweets_name = tweeter_list[i] + '.txt'
    open_path = os.path.join(DATA_DIR, 'tweets', tweets_name)
    with open(open_path, 'w') as f:
        f.write(tweets)

In [202]:
a = m.docvecs.most_similar(positive=open_path, topn=2)

TypeError: '<' not supported between instances of 'str' and 'int'